In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import cftime
import warnings
xr.set_options(keep_attrs=True)
warnings.filterwarnings('ignore')
import sys
sys.path.insert(1, '../')
import my_utils as myf
import importlib
importlib.reload(myf)

<module 'my_utils' from '/gpfsm/dhome/laroach1/analysis/analysis_modelE-so-sst/published/processing/../my_utils.py'>

In [2]:
c6dir = myf.cmip_dir+'CMIP/NASA-GISS/GISS-E2-1-G/historical/'
sspdir = myf.cmip_dir+'ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp245/'

In [3]:
y1, y2 = 1990, 2021

In [4]:
def open_cmip(ens):
    
    ds = xr.open_mfdataset(c6dir+ens+'/SImon/siconca/gn/*/siconca_SImon_GISS-E2-1-G_historical_'+ens+'_*.nc')
    
    filestr = 'historical'
    if os.path.isdir(sspdir+ens):
        ds2 = xr.open_mfdataset(sspdir+ens+'/SImon/siconca/gn/*/siconca_SImon_GISS-E2-1-G_ssp245_'+ens+'_*.nc')
        ds = xr.concat([ds,ds2],dim='time')
        filestr = 'hist+ssp245'
    ds = ds.sel(time=slice(str(y1),str(y2)))
    ds['time'] = ds.indexes['time'].to_datetimeindex()
    ds['names'] = ens
    ds = ds.set_coords('names')
    ds = ds.groupby('time.year').mean(dim='time')

    return ds, filestr

In [5]:
def get_siconca_avg(ds, outfile):
    ds = ds[['siconca']]

    slope, intercept, r_value, p_value, std_err = myf.linregress(ds.year,ds.load(),dim='year')
    slope = slope.siconca.to_dataset(name='siconca_trend')
    slope.siconca_trend.attrs['units'] = '%/year'
    p_value = p_value.siconca.to_dataset(name='siconca_trend_pval')
    p_value.siconca_trend_pval.attrs['units'] = '1'

    clim = ds.siconca.mean(dim='year').to_dataset(name='siconca_climatology')
    ds = xr.merge([slope,p_value, clim])
    ds.to_netcdf(myf.processed_dir+'/spatial_trend/siconca/'+str(y1)+'-'+str(y2)+'/'+outfile+str(y1)+'-'+str(y2)+'.nc')
    
    return ds

In [6]:
runmap = pd.read_csv('../e2.1runmap.csv',sep=',')
allens = runmap['names'].values

In [7]:
ens_ctrl = [f for f in allens if 'f2' in f or 'f1' in f]
ens_mw = [f for f in allens if 'f4' in f]
ens_wind = [f for f in allens if 'f5' in f]
ens_windmw = [f for f in allens if 'f6' in f]

In [8]:
for n, myens in enumerate([ens_ctrl,ens_mw, ens_wind, ens_windmw]):
    listds = []
    print(myens,len(myens))
    for ens in myens:
        ds, filestr = open_cmip(ens)
        listds.append(ds)
    ds = xr.concat(listds,dim='names').mean(dim='names')
    ds['names'] = ['CTRLmean','MWmean','WINDmean','WIND&MWmean'][n]
    ds.attrs['num_ensemble'] = [20,10,3,2][n]
    ds = ds.set_coords('names')
    ds = get_siconca_avg(ds,'ensemble_means/siconca_SImon_GISS-E2-1-G_'+filestr+'_'+str(ds.names.values)+'_gn_')

['r1i1p1f2', 'r2i1p1f2', 'r3i1p1f2', 'r4i1p1f2', 'r5i1p1f2', 'r6i1p1f2', 'r7i1p1f2', 'r8i1p1f2', 'r9i1p1f2', 'r10i1p1f2', 'r201i1p1f1', 'r202i1p1f1', 'r203i1p1f1', 'r204i1p1f1', 'r205i1p1f1', 'r206i1p1f1', 'r207i1p1f1', 'r208i1p1f1', 'r209i1p1f1', 'r210i1p1f1'] 20
['r201i1p1f4', 'r202i1p1f4', 'r203i1p1f4', 'r204i1p1f4', 'r205i1p1f4', 'r206i1p1f4', 'r207i1p1f4', 'r208i1p1f4', 'r209i1p1f4', 'r210i1p1f4'] 10
['r1i1p1f5', 'r201i1p1f5', 'r202i1p1f5'] 3
['r201i1p1f6', 'r202i1p1f6'] 2


In [9]:
for ens in allens[:]:
    ds, filestr = open_cmip(ens)
    ds = get_siconca_avg(ds,'siconca_SImon_GISS-E2-1-G_'+filestr+'_'+ens+'_gn_')

In [10]:
myd = myf.obs_dir+'/CDRv4/siconc/sh_remap1deg/'
myfiles = sorted([f for f in os.listdir(myd) if '.nc' in f])
listds = []
for f in myfiles:
    ds = xr.open_dataset(myd+f)
    listds.append(ds)
ds = xr.concat(listds,dim='tdim')
ds['tdim'] = ds.time
ds = ds.drop('time')
ds = ds.rename({'tdim':'time','cdr_seaice_conc':'siconca','latitude':'lat','longitude':'lon'})
ds = ds.sortby('time')
ds = ds.sel(time=slice('1990','2021'))
ds = ds.groupby('time.year').mean(dim='time')
ds['siconca'] = 100.*ds.siconca
ds['names'] = 'OBS_CDRv4'
outds = get_siconca_avg(ds,str(ds.names.values)+'_')